In [ ]:
pip install pycocotools

In [ ]:
import pandas as pd
from PIL import Image
import requests
from io import BytesIO
from pycocotools.coco import COCO
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
def load_and_proccess_image(id):
  # Load image, then scale and shift pixel values to [-0.5, 0.5]
  id=str(id)
  for i in range(12-len(id)):
    id='0'+id
  img = Image.open('../input/coco2014/train2014/train2014/COCO_train2014_'+id+'.jpg')
  img=img.resize((64,64))
  im = img_to_array(img)
  return im / 255 - 0.5

def read_images(list_url):
  # paths is a dict mapping image ID to image path
  # Returns a dict mapping image ID to the processed image
  ims = []
  for i in range(4000):
    ims.append(load_and_proccess_image(list_url[i]))
  return ims
    
  


## load

In [ ]:
coco=COCO('../input/coco2014/captions/annotations/instances_train2014.json')

In [ ]:
train=pd.read_csv('../input/qa-coco1/train_2.csv')

In [ ]:
train.columns=['question','color','id']

In [ ]:
train_question=train['question'].values
train_answer=train['color'].values
train_id=train['id'].values

In [ ]:
#train_answer=train_answer[:5000]
#train_question=train_question[:5000]
#train_id=train_id[:5000]

In [ ]:
ims=[]
for i in range(len(train_id)):
  id=train_id[i]
  ims.append(load_and_proccess_image(id))


In [ ]:
del_index=[]
for k in range(len(train_id)):
    if ims[k].shape!=(64,64,3):
        del_index.append(k)
        
import cv2
import numpy as np
for j in del_index:
    gray = ims[j]
    ims[j] = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)

In [ ]:
all_answer=np.unique(train_answer)

In [ ]:
train_X_ims = ims

In [ ]:

from tensorflow.keras.utils import to_categorical

# Create model input images


# Create model outputs
train_answer_indices = [all_answer.tolist().index(a) for a in train_answer]

train_Y = to_categorical(train_answer_indices)



In [ ]:
len(train_answer_indices)

# Model

## image

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten
from keras.layers import Activation, Dense

# The CNN
im_input = Input(shape=(64, 64,3))
x1 = Conv2D(8, 3, padding='same')(im_input)
x1 = MaxPooling2D()(x1)
x1 = Conv2D(16, 3, padding='same')(x1)
x1 = MaxPooling2D()(x1)
x1 = Flatten()(x1)
# Add a final fully-connected layer after the CNN for good measure
x1 = Dense(32, activation='tanh')(x1)

## question

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer



# Fit tokenizer on the training questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_question)

# Convert questions to BOW
train_X_seqs = tokenizer.texts_to_matrix(train_question)


# Example BOW:
# [0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
from tensorflow.keras.layers import Input, Dense

# We add one because the Keras Tokenizer reserves index 0 and never uses it.
vocab_size = len(tokenizer.word_index) + 1

# The question network
q_input = Input(shape=(vocab_size,))
x2 = Dense(32, activation='tanh')(q_input)
x2 = Dense(32, activation='tanh')(x2)

## merge

In [ ]:
from tensorflow.keras.layers import Multiply
out = Multiply()([x1, x2])
out = Dense(32, activation='tanh')(out)
# num_answers will be defined below
out = Dense(len(all_answer), activation='softmax')(out)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
model = Model(inputs=[im_input, q_input], outputs=out)
model.compile(
  Adam(lr=2e-4), # somewhat arbitrarily chosen
  loss='categorical_crossentropy',
  metrics=['accuracy'],)

# fit

In [ ]:
import tensorflow as tf
train_X_ims=tf.stack(train_X_ims)


In [ ]:
len(train_id)

In [ ]:
model.fit(
  # Reminder: train_X_seqs is from this post's BOW section
  [train_X_ims[:10000], train_X_seqs[:10000]],
  train_Y[:10000],
  validation_data=([train_X_ims[10000:], train_X_seqs[10000:]], train_Y[10000:]),
  shuffle=True,
  epochs=5,steps_per_epoch=25, validation_steps=25 # somewhat arbitrary, try more epochs if you have time!
)

In [ ]:
model.save('QA.h5')

# test

In [ ]:
import matplotlib.pyplot as plt

id=str(194)
for i in range(12-len(id)):
    id='0'+id
img = Image.open('../input/coco2014/train2014/train2014/COCO_train2014_'+id+'.jpg')
plt.imshow(img)

img=img.resize((64,64))
im = img_to_array(img)
img=im / 255 - 0.5
img=[img]


img=tf.stack(img)

ques=['what is the color of tree']
ques=tokenizer.texts_to_matrix(ques)
print(all_answer[np.argmax(model.predict([img,ques],steps=1), axis=1)[0]])

